<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/k2_pix_small.png">
*This notebook contains an excerpt instructional material from [gully](https://twitter.com/gully_) and the [K2 Guest Observer Office](https://keplerscience.arc.nasa.gov/); the content is available [on GitHub](https://github.com/gully/k2-metadata).*


<!--NAVIGATION-->
< [Advanced visualizations](04.00-Advanced-visualizations.ipynb) | [Contents](Index.ipynb) | [Deprecated content](05.00-Deprecated-content.ipynb) >

# Explore the EPIC Catalog

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.utils.console import ProgressBar
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import feather

In [4]:
path = '../metadata/EPIC_catalog/EPIC_Catalog_16cols.feather'

In [5]:
%time df = feather.read_dataframe(path)

CPU times: user 15.9 s, sys: 17.4 s, total: 33.3 s
Wall time: 1min 7s


```
CPU times: user 16.2 s, sys: 13.3 s, total: 29.5 s
Wall time: 50 s
```
*Feather is about 8.5x faster than reading the csv files individually.*

In [7]:
df.shape

(50545738, 16)

In [8]:
df.columns

Index(['id', 'hip', 'tyc', 'ucac', 'twomass', 'sdss', 'objtype', 'kepflag',
       'stpropflag', 'k2_ra', 'k2_dec', 'kp', 'nomad', 'mflg', 'prox',
       'k2_avail_flag'],
      dtype='object')

In [9]:
df.head()

,id,hip,tyc,ucac,twomass,sdss,objtype,kepflag,stpropflag,k2_ra,k2_dec,kp,nomad,mflg,prox,k2_avail_flag
0,201000001.0,NaN,None,None,11370387-1048329,None,STAR,JHK,None,174.266139,-10.809154,18.558001,None,None,NaN,False
1,201000002.0,NaN,None,None,11372639-1048232,None,STAR,J,None,174.359970,-10.806450,18.257000,None,None,NaN,False
2,201000003.0,NaN,None,396-052344,11390735-1048225,None,STAR,BV,pmV,174.780561,-10.806233,16.306000,None,None,NaN,False
3,201000004.0,NaN,None,None,11372709-1048189,None,STAR,JHK,None,174.362902,-10.805261,18.348000,None,None,NaN,False
4,201000005.0,NaN,None,396-052314,11380583-1048162,None,STAR,gri,pmV,174.524379,-10.804541,15.894000,None,None,NaN,False


## Datashader

In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

In [12]:
import datashader as ds
from datashader.bokeh_ext import InteractiveImage

/Users/obsidian/anaconda3/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [13]:
def base_plot():
    p = figure(
        x_range=(0, 360),
        y_range=(-90, 90),
        tools='pan,wheel_zoom,box_zoom,reset', 
        plot_width=1000, 
        plot_height=500,
    )
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.axis_label = "RA"
    p.yaxis.axis_label = "DEC"
    p.xaxis.axis_label_text_font_size = '12pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    return p
    
options = dict(line_color=None, fill_color='blue', size=5)

In [16]:
df['flux'] = 10**(df.kp/(-2.5))

In [18]:
p = base_plot()
pipeline = ds.Pipeline(df, ds.Point("k2_ra", "k2_dec"), agg=ds.reductions.sum('flux'), )
InteractiveImage(p, pipeline)

In [ ]:
import numpy as np
import holoviews as hv
from holoviews.operation.datashader import aggregate
hv.notebook_extension('bokeh')

%opts Curve [width=800]

dataset = hv.Dataset(df, kdims=['flux', 'kp'], vdims=[]).select(kp=(0,25))
agg = aggregate(dataset, aggregator=ds.count(), streams=[hv.streams.RangeX()], x_sampling=0.5, width=500, height=2)
result = agg.map(lambda x: x.reduce(flux=np.sum), hv.Image)

## Simple datashader

In [12]:
import datashader as ds
import datashader.transfer_functions as tf
import pandas as pd

In [13]:
cvs = ds.Canvas(plot_width=1400, plot_height=1400)

In [14]:
agg = cvs.points(df, 'k2_ra', 'k2_dec')

In [15]:
img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='log')

In [24]:
img.shape

(1400, 1400)

In [1]:
#plt.figure(figsize=(10,10))
#plt.imshow(img)

The EPIC catalog was generated by projecting anticipated fields onto the sky.

<!--NAVIGATION-->
< [Advanced visualizations](04.00-Advanced-visualizations.ipynb) | [Contents](Index.ipynb) | [Deprecated content](05.00-Deprecated-content.ipynb) >